# Hotel Recommendation System(ML Assignment)


In [1]:
%pip install keras.layers.core

ERROR: Could not find a version that satisfies the requirement keras.layers.core (from versions: none)
ERROR: No matching distribution found for keras.layers.core
Note: you may need to restart the kernel to use updated packages.


# COD3

In [1]:
pip install pipreqs
pipreqs .
pip install -r requirements.txt

SyntaxError: invalid syntax (3763389659.py, line 1)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
# from keras.layers.core import Dense,Activation,Dropout
# from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
hotel_details=pd.read_excel('/content/drive/My Drive/hotel.xlsx')


In [ ]:
hotel_rooms=pd.read_excel('/content/drive/My Drive/rooms_hotel.xlsx')

In [ ]:
events_df = pd.read_excel('/content/drive/My Drive/event.xlsx')

# EDA

In [ ]:
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()

In [ ]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotel_id',right_on='id',how='inner')

In [ ]:
def check_missing_values(dataset):
    missing_values = dataset.isnull().sum()
    total_values = dataset.shape[0]

    if missing_values.sum() == 0:
        print("В датасете нет пропущенных значений.")
    else:
        print("Пропущенные значения в датасете:")
        missing_data = missing_values[missing_values > 0]
        missing_data_percentage = (missing_data / total_values) * 100
        missing_data_summary = pd.concat([missing_data, missing_data_percentage], axis=1)
        missing_data_summary.columns = ['Количество пропусков', 'Процент пропусков']
        print(missing_data_summary)

check_missing_values(hotel)


В датасете нет пропущенных значений.


In [ ]:

hotel = hotel.merge(events_df, on='hotel_id', how='left')

In [ ]:
hotel.columns

Index(['room_id_x', 'hotel_id', 'category', 'decription', 'ratedescription',
       'price', 'category_rus', 'max_count', 'room_img', 'id', 'name',
       'address', 'city', 'country', 'country_rus', 'propertytype', 'stars',
       'hotel_img', 'visitorid', 'room_id_y', 'status', 'status_name',
       'cluster'],
      dtype='object')

In [ ]:
del hotel['room_id_y']
del hotel['hotel_id']


In [ ]:
hotel.rename(columns={'room_id_x': 'room_id'}, inplace=True)

hotel.rename(columns={'id': 'hotel_id'}, inplace=True)


# Рекомендательная система, основанная только на городе и рейтингах отеля

In [ ]:
hotel['visitorid'].fillna(0, inplace=True)
hotel['hotel_id'].fillna(0, inplace=True)
hotel.fillna(0, inplace=True)
hotel['interaction_value'] = hotel['status_name'].apply(lambda x: 0.6 if x == 'order' else (0.8 if x == 'order' else 0.4))

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix


# hotel['interaction_value'] = hotel['status_name'].apply(lambda x: 0.6 if x == 'order' else (0.8 if x == 'order' else 0.4))

item_index_map = {item: idx for idx, item in enumerate(hotel['hotel_id'].unique())}
visitor_index_map = {visitor: idx for idx, visitor in enumerate(hotel['hotel_id'].unique())}
cluster_index_map = {cluster: idx for idx, cluster in enumerate(hotel['cluster'].unique())}

row_indices = [visitor_index_map[visitor] for visitor in hotel['hotel_id']]
col_indices = [item_index_map[item] for item in hotel['hotel_id']]
interaction_values = hotel['interaction_value']

interaction_matrix = csr_matrix((interaction_values, (row_indices, col_indices)),
                                 shape=(len(visitor_index_map), len(item_index_map)))

# к матрице interaction_matrix добавляется информация о кластерах (clusters). Для каждого события значение взаимодействия умножается на 1 + 0.1 * индекс кластера, чтобы отразить влияние кластера на взаимодействие.
cluster_indices = [cluster_index_map[cluster] for cluster in hotel['cluster']]
for i in range(len(interaction_values)):
    interaction_matrix[row_indices[i], col_indices[i]] *= 1 + 0.1 * cluster_indices[i]  # Adjust interaction value based on cluster

print(interaction_matrix)

# код выводит разреженную матрицу interaction_matrix,
# которая представляет собой матрицу взаимодействия между посетителями (строки) и товарами (столбцы)
# на основе значений в столбце 'interaction_value

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Вычисление сходства пользователей
user_similarity = cosine_similarity(interaction_matrix)

# Создание функции для получения рекомендаций для конкретного пользователя
def get_recommendations(visitorid, num_recommendations=5):
    # Находим наиболее похожих пользователей
    similar_users = user_similarity[visitorid]
    similar_users_idx = similar_users.argsort()[::-1][1:]  # Исключаем самого пользователя

    # Получаем товары, с которыми взаимодействовали похожие пользователи
    recommendations = set()
    for similar_user_idx in similar_users_idx:
        similar_user_actions = interaction_matrix[similar_user_idx]
        new_recommendations = set(hotel.loc[similar_user_actions > 0, 'itemid'])
        recommendations.update(new_recommendations)
        if len(recommendations) >= num_recommendations:
            break

    # Исключаем товары, с которыми пользователь уже взаимодействовал
    user_actions = interaction_matrix[visitorid]
    recommendations.difference_update(set(hotel.loc[user_actions > 0, 'itemid']))

    return list(recommendations)[:num_recommendations]

In [ ]:
%pip install scikit-surprise

In [ ]:
hotel[['visitorid', 'itemid', 'interaction_value']]

,visitorid,itemid,interaction_value
0,902806.0,46487,0.4
1,141441.0,46487,0.4
2,867351.0,46487,0.4
3,428686.0,46487,0.4
4,649742.0,46487,0.4
...,...,...,...
9288,0.0,47125,0.8
9289,0.0,47125,0.8
9290,0.0,47125,0.8
9291,0.0,47125,0.8


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(hotel[['visitorid', 'itemid', 'interaction_value']], reader)

param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.02, 0.1, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['RMSE', 'MAE'], cv=5)
gs.fit(data)

print(gs.best_params['rmse'])
print(gs.best_params['mae'])

model = SVD(n_epochs=15, lr_all=0.01, reg_all=0.02)
cross_validate(model,  data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()
model.fit(trainset)

TypeError: Singleton array array(<surprise.dataset.DatasetAutoFolds object at 0x78d563f35b10>,
      dtype=object) cannot be considered a valid collection.

In [ ]:
from surprise.dump import dump

model_filename = 'trained_model.pkl'
dump(model_filename, algo=model)

In [ ]:
from surprise.dump import load

loaded_model = load(model_filename)[1]

In [ ]:
def get_top_n_recommendations(model, user_id, num_items, n=10):
    recommendations = []
    for item_id in range(1, num_items + 1):
        prediction = model.predict(user_id, item_id)
        recommendations.append((item_id, prediction.est))

    # Sort recommendations by estimated rating
    recommendations.sort(key=lambda x: x[1], reverse=True)

    top_n_recommendations = recommendations[:n]
    return top_n_recommendations

user_id = 10
num_items = hotel['itemid'].nunique()
top_recommendations = get_top_n_recommendations(loaded_model, user_id, num_items, n=5)
print("Лучшие рекомендации для пользователей", user_id)
for item_id, estimated_rating in top_recommendations:
    print("Item:", item_id, "Предполагаемый рейтинг по продукту:", estimated_rating)


In [ ]:
user_id = 200
num_items = events_df['itemid'].nunique()
top_recommendations = get_top_n_recommendations(loaded_model, user_id, num_items, n=10)
print("Лучшие рекомендации для пользователей", user_id)
for item_id, estimated_rating in top_recommendations:
    print("Item:", item_id, "Предполагаемый рейтинг по продукту:", estimated_rating)

In [ ]:
hotel['decription']=hotel['decription'].str.replace(': ;',',')

In [ ]:
hotel.head()

,room_id,category,decription,ratedescription,price,category_rus,max_count,room_img,hotel_id,name,...,city,country,country_rus,propertytype,stars,hotel_img,visitorid,status,status_name,cluster
0,7686,Superior Double Bed,"air conditioning,alarm clock,blackout curtains...","Room size: 25 mВІ/269 ftВІ, Garden view, Balco...",16872,Улучшенный номер с двуспальной кроватью,2,https://encrypted-tbn0.gstatic.com/images?q=tb...,989,Hotel Rural Alves - Casa Alves Torneiros,...,Penso,portugal,Португалия,Hotels,4,https://www.menslife.com/upload/images/2019/10...,NaN,NaN,NaN,NaN
1,7666,Superior Triple Room,"air conditioning,alarm clock,carpeting,closet,...","Room size: 25 mВІ/269 ftВІ, Balcony/terrace, S...",15827,Улучшенный трехместный номер,3,https://encrypted-tbn0.gstatic.com/images?q=tb...,989,Hotel Rural Alves - Casa Alves Torneiros,...,Penso,portugal,Португалия,Hotels,4,https://www.menslife.com/upload/images/2019/10...,NaN,NaN,NaN,NaN
2,7667,Superior Room With 2 Single Beds,"air conditioning,free wi-fi in all rooms!,in-r...","Room size: 26 mВІ/280 ftВІ, 2 single beds and ...",16650,Улучшенный номер с 2 односпальными кроватями,1,https://encrypted-tbn0.gstatic.com/images?q=tb...,989,Hotel Rural Alves - Casa Alves Torneiros,...,Penso,portugal,Португалия,Hotels,4,https://www.menslife.com/upload/images/2019/10...,NaN,NaN,NaN,NaN
3,7672,Superior Room With 2 Single Beds,"air conditioning,free wi-fi in all rooms!,in-r...","Room size: 26 mВІ/280 ftВІ, 2 single beds and ...",18243,Улучшенный номер с 2 односпальными кроватями,1,https://encrypted-tbn0.gstatic.com/images?q=tb...,989,Hotel Rural Alves - Casa Alves Torneiros,...,Penso,portugal,Португалия,Hotels,4,https://www.menslife.com/upload/images/2019/10...,NaN,NaN,NaN,NaN
4,7674,Superior Room With 2 Single Beds,"air conditioning,free wi-fi in all rooms!,in-r...","Room size: 26 mВІ/280 ftВІ, 2 single beds and ...",24242,Улучшенный номер с 2 односпальными кроватями,1,https://encrypted-tbn0.gstatic.com/images?q=tb...,989,Hotel Rural Alves - Casa Alves Torneiros,...,Penso,portugal,Португалия,Hotels,4,https://www.menslife.com/upload/images/2019/10...,NaN,NaN,NaN,NaN


In [ ]:
def requirementbased(country,number,features):
    hotel['country']=hotel['country'].str.lower()
    hotel['decription']=hotel['decription'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['country']==country.lower()]
    reqbased=reqbased[reqbased['max_count']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['decription'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotel_id',keep='first',inplace=True)
    return reqbased[['name','country_rus','stars','price','hotel_img','room_img','decription','category']].head(10)

# Рекомендации, основанные на требованиях и особых потребностях

In [ ]:
hotel.to_excel (r'/content/drive/My Drive/Hotel_itog.xlsx', index= False )

In [ ]:
hotel_img = pd.read_excel('/content/drive/My Drive/картинки отелей.xlsx')

In [ ]:
hotel = pd.merge(hotel, hotel_img, on='id', how='inner')

In [ ]:
room_img = pd.read_excel('/content/drive/My Drive/картинки_комнат.xlsx')

In [ ]:
hotel = pd.merge(hotel, room_img, on='id', how='inner')

In [ ]:
hotel.to_excel (r'/content/drive/My Drive/Hotel_itog_new.xlsx', index= False )

In [ ]:
import pickle
Pkl_Filename = "Pickle_RL_Model.pkl"
with open(Pkl_Filename, 'wb') as file:
    pickle.dump(requirementbased, file)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
requirementbased('Italy',1,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

In [ ]:
hotel.head()

,hotelcode_x,roomamenities,roomtype,ratedescription,hotelname,address,city,country,propertytype,starrating,latitude,longitude,hotelcode_y,guests_no
0,31,"air conditioning,alarm clock,bathrobes,closet,...",Classic Double or Twin Room,"Room size: 20 mВІ/215 ftВІ, City view, Non-smo...",Hotel Alpenrose,Postfach,Riederalp,switzerland,Hotels,3,46.379391,8.03577,2409,2
1,31,"air conditioning,alarm clock,bathrobes,closet,...",Classic Double or Twin Room,"Room size: 20 mВІ/215 ftВІ, City view, Non-smo...",Hotel Alpenrose,Postfach,Riederalp,switzerland,Hotels,3,46.379391,8.03577,2409,2
2,31,"air conditioning,alarm clock,bathrobes,closet,...",Classic Double or Twin Room,"Room size: 20 mВІ/215 ftВІ, City view, Non-smo...",Hotel Alpenrose,Postfach,Riederalp,switzerland,Hotels,3,46.379391,8.03577,2409,2
3,31,"air conditioning,alarm clock,bathrobes,closet,...",Classic Double or Twin Room,"Room size: 20 mВІ/215 ftВІ, City view, Non-smo...",Hotel Alpenrose,Postfach,Riederalp,switzerland,Hotels,3,46.379391,8.03577,2409,2
4,31,"air conditioning,alarm clock,bathrobes,closet,...",Classic Double or Twin Room,"Room size: 20 mВІ/215 ftВІ, City view, Non-smo...",Hotel Alpenrose,Postfach,Riederalp,switzerland,Hotels,3,46.379391,8.03577,2409,2


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Пример функции модели на Python
def predict(input_data):
    # Здесь должен быть код вашей модели, который обрабатывает входные данные и возвращает результат
    return "Модель вернула результат для входных данных: " + input_data

@app.route('/predict', methods=['GET'])
def get_prediction():
    # input_data = request.json.get('input_data')
    result = requirementbased('Italy',1,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')
    return jsonify({'result': result})

if __name__ == '__main__':
    app.run(debug=True)

const inputData = 'Ваши входные данные';
fetch('http://localhost:5000/predict', {
    method: 'GET',
    headers: {
        'Content-Type': 'application/json'
    },
    body: JSON.stringify({ input_data: inputData })
})
.then(response => response.json())
.then(data => {
    console.log(data.result);
});


In [ ]:
!git clone https://github.com/vTuanpham/Large_dataset_translator.git

%cd Large_dataset_translator

%pip install -r requirements.txt

In [ ]:
%run examples/YahmaAlpaca/AlpacaCleaned_Parser.py

Чтобы преобразовать свой собственный набор данных:
Унаследуйте класс DataParser и реализуйте свою логику чтения и преобразования.

Убедитесь, что функция преобразования сопоставляет все поля из исходного набора данных с полями в configs/base_config.py, или выберите другие конфигурации, соответствующие вашему набору данных.

Установите do_translate=True в супер-вызове, чтобы включить перевод.

 def __init__(self, file_path: str, output_path: str):
 super().__init__(file_path, output_path,
имя_парасера=ИМЯ_ПАРАСЕРА,
do_translate=True,
no_translated_code=True,
target_lang="ko")
Настройте параметры перевода, такие как язык перевода и фильтры предварительного перевода.

In [ ]:
from google.colab import files

hotel.to_csv('hotel.csv')
files.download('hotel.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from configs.base_config import BaseConfig

class HotelDataConverter(BaseConfig):
    def __init__(self, file_path: str, output_path: str):
        super().__init__(file_path, output_path,
                         parser_name="hotel",
                         do_translate=True,
                         no_translated_code=True,
                         target_lang="ru")

In [ ]:
from dataclasses import dataclass

@dataclass
class BaseConfig:
    file_path: str
    output_path: str
    do_translate: bool

    def __post_init__(self):
        self.file_name = self.file_path.split("/")[-1]
        self.output_file_path = f"{self.output_path}/{self.file_name}"

    def get_dict(self):
        return self.__dict__

    def get_keys(cls):
        return list(cls.__annotations__.keys())

class HotelDataConverter(BaseConfig):
    def __init__(self, file_path: str, output_path: str):
        super().__init__(file_path, output_path,
                         parser_name="hotel",
                         do_translate=True,
                         no_translated_code=True,
                         target_lang="ru")

        # Дополнительная логика для класса HotelDataConverter

In [ ]:
converter = HotelDataConverter("path/to/input_file.csv", "path/to/output_folder/")
converter.convert()


TypeError: BaseConfig.__init__() got an unexpected keyword argument 'parser_name'

In [ ]:

hotel
hotel.read_data()
hotel.transform_data()

AttributeError: 'DataFrame' object has no attribute 'read_data'

In [ ]:
def __init__(self, file_path: str, output_path: str):
  super().__init__(file_path, output_path,
                    parser_name=PARSER_NAME,
                     do_translate=True,
                     no_translated_code=True,
                     target_lang="ru")